In [6]:
%pip install seaborn scipy scikit-learn pandas numpy matplotlib ipywidgets

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter, detrend
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import glob
import ipywidgets as widgets
from IPython.display import display

# Configuração visual
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
def aplicar_snv(espectros):
    """Standard Normal Variate (Remove efeito de espalhamento de luz)"""
    media = np.mean(espectros, axis=1, keepdims=True)
    desvio = np.std(espectros, axis=1, keepdims=True)
    return (espectros - media) / desvio

def aplicar_derivada(espectros, janela=15, ordem=2, deriv=1):
    """Filtro Savitzky-Golay"""
    return savgol_filter(espectros, window_length=janela, polyorder=ordem, deriv=deriv, axis=1)

def aplicar_alisamento(espectros, janela=15, ordem=2):
    """Apenas suavização (Derivada 0)"""
    return savgol_filter(espectros, window_length=janela, polyorder=ordem, deriv=0, axis=1)

def aplicar_detrend(espectros):
    """Remove tendência linear (baseline inclinada)"""
    return detrend(espectros, axis=1)%pip install seaborn scipy scikit-learn pandas numpy matplotlib ipywidgets

def processar_dados(X, pipeline_steps):
    """
    Aplica uma sequência de tratamentos definidos pelo usuário.
    """
    X_temp = X.copy()
    for passo in pipeline_steps:
        tipo = passo['tipo']
        
        if tipo == 'SNV':
            X_temp = aplicar_snv(X_temp)
            print(" -> Aplicado SNV")
            
        elif tipo == 'Derivada':
            win = passo.get('janela', 15)
            poly = passo.get('polinomio', 2)
            der = passo.get('ordem', 1)
            X_temp = aplicar_derivada(X_temp, win, poly, der)
            print(f" -> Aplicado Savitzky-Golay (Janela: {win}, Derivada: {der})")
            
        elif tipo == 'Detrend':
            X_temp = aplicar_detrend(X_temp)
            print(" -> Aplicado Detrend")
            
    return X_temp

In [ ]:
# --- CÉLULA 3: Configuração Dinâmica ---

# 1. Encontrar arquivos automaticamente na pasta
arquivos_txt = glob.glob('*.txt') # Ou coloque o caminho ex: 'dados/*.txt'

if not arquivos_txt:
    print("⚠️ Nenhum arquivo .txt encontrado na pasta!")
    arquivos_txt = ['Simulacao_Glicose.txt'] # Fallback

# Widgets para seleção
seletor_abs = widgets.Dropdown(options=arquivos_txt, description='Absorbância:')
seletor_conc = widgets.Dropdown(options=arquivos_txt, description='Concentração:')

print("📂 Selecione os arquivos para análise:")
display(seletor_abs)
display(seletor_conc)

# --- 🛠️ CONFIGURAÇÃO DO PIPELINE (Aqui você define a ordem) ---
# Você pode adicionar quantos passos quiser. O código executa de cima para baixo.
PIPELINE = [
    # Passo 1: Remover espalhamento
    {'tipo': 'SNV'}, 
    
    # Passo 2: Remover baseline inclinada
    # {'tipo': 'Detrend'}, 
    
    # Passo 3: Ressaltar picos
    {'tipo': 'Derivada', 'janela': 17, 'polinomio': 2, 'ordem': 1} 
]

CONFIG_MODELO = {
    'max_componentes': 10,
    'folds': 5,
    'analito_coluna': 0
}

⚠️ Nenhum arquivo .txt encontrado na pasta!
📂 Selecione os arquivos para análise:


Dropdown(description='Absorbância:', options=('Simulacao_Glicose.txt',), value='Simulacao_Glicose.txt')

Dropdown(description='Concentração:', options=('Simulacao_Glicose.txt',), value='Simulacao_Glicose.txt')